###### Setting up Environment variables

In [1]:
import sys
sys.path.append('/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/')
sys.path.append('./')
print(sys.version)

3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]


In [2]:
###########import necessary keys 

import os
#import openai
from secret import OPENAI_API_TYPE,OPENAI_API_KEY,OPENAI_API_BASE,HUGGINGFACEHUB_API_TOKEN
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# openai.api_version = "2023-07-01-preview"

os.environ['OPENAI_API_TYPE']=OPENAI_API_TYPE
os.environ['OPENAI_API_BASE']=OPENAI_API_BASE
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN






###### Set & Identify right LLM Model & Langchain Module 

In [3]:
##### Unstable connectivity between langchain and openai. Better to test which link is working an any given point in time. 

os.environ['OPENAI_API_VERSION'] = "2023-08-01-preview" #"2023-05-15" #"2022-12-01"
deployment_list =["deploy-name-embed","deploy-gpt35turbo16k", "datachatdeploy"]
model_list=["text-embedding-ada-002","gpt-35-turbo-16k", "text-davinci-003","gpt-35-turbo"]

for deploy_name in deployment_list:
    for mod_name in model_list:
        try:
            llm = AzureOpenAI(deployment_name=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : AzureOpenAI")
                break
        except Exception as e:
            print(f"Error with AzureOpenAI:{e}\n" )
        
        try:
            llm = AzureChatOpenAI(deployment_name=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : AzureOpenAIChat")
                break
        except Exception as e:
            print(f"Error with AzureOpenAIChat:{e}\n" )
        
        try:
            llm= OpenAI(engine=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : OpenAI")
                break
        except Exception as e:
            print(f"Error with OpenAI:{e}\n" )

        try:
            llm= OpenAIChat(engine=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : OpenAIChat")
                break
        except Exception as e:
            print(f"Error with OpenAIChat:{e}\n" )




AzureOpenAI
Params: {'deployment_name': 'deploy-name-embed', 'model_name': 'text-embedding-ada-002', 'temperature': 0.0, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}
Error with AzureOpenAI:The completion operation does not work with the specified model, text-embedding-ada-002. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='text-embedding-ada-002' temperature=0.0 model_kwargs={} openai_api_key='ff0b3bcd1b9843e2bf0c172b531cb52a' openai_api_base='https://ws-openai-ml-aoai.openai.azure.com/' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=No

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/utils/utils.py:157: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
print(llm)

OpenAIChat
Params: {'model_name': 'text-embedding-ada-002', 'engine': 'datachatdeploy', 'temperature': 0}


In [6]:
llm("what is esg?") 

"ESG stands for Environmental, Social, and Governance. It is a framework used to evaluate the sustainability and ethical impact of a company or investment. \n\nEnvironmental factors refer to a company's impact on the environment, such as its carbon emissions, waste management, and resource usage. Social factors consider a company's impact on society, including its treatment of employees, community engagement, and diversity and inclusion practices. Governance factors assess the company's leadership, board structure, executive compensation, and transparency in decision-making.\n\nESG criteria are used by investors, financial institutions, and stakeholders to assess the long-term sustainability and ethical practices of a company. It helps them make informed decisions about investments, as well as encourages companies to adopt responsible business practices."

#### Load the document file that will be vectorized and indexed.

In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

loader = PyPDFLoader("./data/files/esg_strategy.pdf")
pages = loader.load_and_split() ## this split the doc by pdf pages.

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)


In [30]:
print("size of the doc, that is elements :",len(pages))
print(pages[0])
print()
print(pages[1])

size of the doc, that is elements : 5
page_content="Impact  of ESG Strategy  on Investment  Management  and Asset  Management:  A Case  Study  \non Northern  Trust  \nExecutive  Summary:  Environmental, Social, and Governance (ESG) factors have \nsignificantly influenced the investment landscape in recent years. For investment \nmanagement and asset management companies, including Northern Trust, the \nincorporation of ESG criteria can drive busi ness decisions, shape investment portfolios, \nand influence stakeholder relations. This document delves into the impact of ESG \nstrategies on these institutions.  \n1. Introduction:  \n• ESG Strategy : A set of investment criteria that consider environmental, social, \nand governance factors alongside financial factors in the investment decision -\nmaking process.  \n• Northern  Trust : An international investment management and asset \nmanagement company that has integrated ESG strategies into its products and \nservices.  \n2. Direct  Impa

In [29]:
# this split doc by lines or sentences.
from langchain.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("data/esg_strategy.pdf",mode='elements') # this split doc by lines or sentences.
data = loader.load()
print("size of the doc, that is elements :",len(data))
print(data[0])
print()
print(data[1])



size of the doc, that is elements : 102
page_content='Impact of ESG Strategy on Investment Management and Asset Management: A Case Study on Northern Trust' metadata={'source': 'data/esg_strategy.pdf', 'coordinates': {'points': ((72.264, 90.65840000000003), (72.264, 117.01999999999998), (541.70704, 117.01999999999998), (541.70704, 90.65840000000003)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'filename': 'esg_strategy.pdf', 'file_directory': 'data', 'last_modified': '2023-09-19T14:56:32', 'filetype': 'application/pdf', 'page_number': 1, 'category': 'UncategorizedText'}

page_content='Executive Summary: Environmental, Social, and Governance (ESG) factors have significantly influenced the investment landscape in recent years. For investment management and asset management companies, including Northern Trust, the incorporation of ESG criteria can drive business decisions, shape investment portfolios, and influence stakeholder relations. This document delves into t

###### Handling multiple pdf,docs& text files for embeddings

In [ ]:
## handle multiple documents
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

documents = []
for file in os.listdir('docs'):
    if file.endswith('.pdf'):
        pdf_path = './docs/' + file
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = './docs/' + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = './docs/' + file
        loader = TextLoader(text_path)
        documents.extend(loader.load())

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

# we now proceed as earlier, passing in the chunked_documents to the
# to the vectorstore
# ...

In [35]:
embeddings = OpenAIEmbeddings(deployment="deploy-name-embed",
    model="text-embedding-ada-002",
    openai_api_base=OPENAI_API_BASE,
    openai_api_type="azure",chunk_size=16,)

print(embeddings)

In [32]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = DirectoryLoader('./data/files/',  show_progress=True,use_multithreading=True) ### parameter to control which files: glob="**/*.pdf",
docs=loader.load()

print(len(docs))

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=10)
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs)


100%|██████████| 3/3 [00:25<00:00,  8.59s/it]


3


In [42]:
returned_embeddings = embeddings.embed_documents(
    [doc.page_content for doc in chunked_documents],
)


doc_embed_db = FAISS.from_embeddings(
    text_embeddings=[
        (doc.page_content, embedding)
        for doc, embedding in zip(chunked_documents, returned_embeddings)
    ], 
    embedding=embeddings,
    metadatas=[
        doc.metadata
        for doc in chunked_documents
    ],
)

query="what is Northern Trust approach for ESG?"

#query_result = embeddings.embed_query(query)
#query_result ### returns vector embeddings

In [43]:
## Saving the embeddings 

doc_embed_db.save_local("data/faiss_index")
#new_db = FAISS.load_local("faiss_index", embeddings)

In [63]:
### For retrieving the document 
##retriever = FAISS.from_documents(texts, embeddings).as_retriever()

##docs = retriever.get_relevant_documents("what is Northern Trust approach for ESG?")
##pretty_print_docs(docs)

Document 1:

execution of their ESG strategies. Through these questions, Northern Trust ensures that 
its investment decisions align with sustainability and responsible business practices, 
reflecting the evolving demands of modern investors.
----------------------------------------------------------------------------------------------------
Document 2:

portfolio optimization and client growth, as well as indirect benefits like enhanced 
reputation and stakeholder engagement.  
By understanding and actively incorporating ESG strategies, investment and asset 
management companies position themselves for sustainable growth and long -term 
success in a rapidly evolving financial landscape.  
 
 
Northern  Trust's  ESG Inquiry  Framework  for Companies  
Executive  Summary:  Northern Trust actively engages with companies to assess their 
adherence to and integration of Environmental, Social, and Governance (ESG) principles. 
This document provides an outline of the typical questions North

In [46]:
doc_embed_db.similarity_search(query)

[Document(page_content='Is the company actively working to decrease the frequency of health and safety issues?\n\nKey Benefits\n\n1\n\nEnables more purposeful and transparent integration of ESG considerations into investment processes, addressing the need for a consistent way to measure and report on ESG investments.\n\n2\n\nProvides a consistent, transparent methodology to gain greater clarity when building and managing sustainable portfolios by focusing on the magnitude and direction of key ESG-related business issues likely to have a financial impact on companies.\n\n3\n\nOffers more precision in identifying ESG-related business risks that could impact its performance, whether constructing portfolios using best-in- class security selection or exclusionary techniques.\n\n4\n\nAssists in prioritizing corporate engagements, further aligning sustainable investments and stewardship — leading to more deliberate engagements and the ability to track progress over time with a consistent metr

In [51]:
## compression of results to address output token limit issue
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

retriever =doc_embed_db.as_retriever()
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)


Document 1:

Enables more purposeful and transparent integration of ESG considerations into investment processes, addressing the need for a consistent way to measure and report on ESG investments.
Provides a consistent, transparent methodology to gain greater clarity when building and managing sustainable portfolios by focusing on the magnitude and direction of key ESG-related business issues likely to have a financial impact on companies.
Offers more precision in identifying ESG-related business risks that could impact its performance, whether constructing portfolios using best-in- class security selection or exclusionary techniques.
Assists in prioritizing corporate engagements, further aligning sustainable investments and stewardship — leading to more deliberate engagements and the ability to track progress over time with a consistent metric.
Best-in-class ESG is industry terminology referring to an investment approach that selects companies that are leaders in implementing ESG.
---

In [63]:
# EmbeddingsFilter
# Making an extra LLM call over each retrieved document is expensive and slow. The EmbeddingsFilter provides a cheaper and faster option by embedding the documents and query and only returning those documents which have sufficiently similar embeddings to the query.

from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.document_compressors import EmbeddingsFilter
import util.helper as helper



embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents(query)
helper.pretty_print_docs(compressed_docs)

Document 1:

Is the company actively working to decrease the frequency of health and safety issues?

Key Benefits

1

Enables more purposeful and transparent integration of ESG considerations into investment processes, addressing the need for a consistent way to measure and report on ESG investments.

2

Provides a consistent, transparent methodology to gain greater clarity when building and managing sustainable portfolios by focusing on the magnitude and direction of key ESG-related business issues likely to have a financial impact on companies.

3

Offers more precision in identifying ESG-related business risks that could impact its performance, whether constructing portfolios using best-in- class security selection or exclusionary techniques.

4

Assists in prioritizing corporate engagements, further aligning sustainable investments and stewardship — leading to more deliberate engagements and the ability to track progress over time with a consistent metric.

Best-in-class ESG is ind

###### Adding Chat History

```
ConversationalRetrievalChain


from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(),
    vectordb.as_retriever(search_kwargs={'k': 6}),
    return_source_documents=True
)
```

#### CSV Data handling

In [26]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path="./data/files/esgdatakaggle/ESGCountry.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',  
        #"fieldnames": ["Country Code","Short Name","Table Name","Long Name","2-alpha code","Currency Unit","Special Notes","Region","Income Group","WB-2 code","National accounts base year","National accounts reference year","SNA price valuation","Lending category","Other groups","System of National Accounts","Alternative conversion factor","PPP survey year","Balance of Payments Manual in use","External debt Reporting status","System of trade","Government Accounting concept","IMF data dissemination standard","Latest population census","Latest household survey","Source of most recent Income and expenditure data","Vital registration complete","Latest agricultural census","Latest industrial data","Latest trade data"],
    },
)

data = loader.load()

#for document in data:
#    content = document.page_content
#    metadata = document.metadata


In [5]:
import pandas as pd
import numpy as np
import random
#### generate sample data for testing
# 1. Investor Information
investors_data = {
    'investor_id': [i for i in range(1, 51)],
    'investor_name': ['Investor ' + str(i) for i in range(1, 51)],
    'contact_info': ['investor' + str(i) + '@example.com' for i in range(1, 51)],
    'address': ['Address ' + str(i) for i in range(1, 51)],
    'account_open_date': pd.date_range(start='2019-01-01', periods=50, freq='15D').strftime('%Y-%m-%d').tolist()
}
investors_df = pd.DataFrame(investors_data)
print("investors_df")
# 2. Asset Information
assets_data = {
    'asset_id': [i for i in range(1, 201)],
    'investor_id': [random.choice(range(1, 51)) for _ in range(200)],
    'asset_type': random.choices(['stocks', 'bonds', 'real estate', 'mutual fund'], k=200),
    'asset_name': ['Asset ' + str(i) for i in range(1, 201)],
    'purchase_date': pd.date_range(start='2019-01-01', periods=200, freq='3D').strftime('%Y-%m-%d').tolist(),
    'purchase_price': [round(random.uniform(1000, 10000), 2) for _ in range(200)],
    'current_value': [round(random.uniform(1000, 10000), 2) for _ in range(200)],
    'currency': ['USD' for _ in range(200)]
}
assets_df = pd.DataFrame(assets_data)
print("assets_df")

# 3. Transaction Information
transactions_data = {
    'transaction_id': [i for i in range(1, 500001)],
    'investor_id': [random.choice(range(1, 51)) for _ in range(500000)],
    'asset_id': [random.choice(range(1, 201)) for _ in range(500000)],
    'transaction_type': random.choices(['buy', 'sell'], k=500000),
    'transaction_date': pd.date_range(start='2019-01-01', periods=500000, freq='H').strftime('%Y-%m-%d').tolist(),
    'transaction_amount': [round(random.uniform(1, 100), 2) for _ in range(500000)],
    'transaction_price': [round(random.uniform(100, 1000), 2) for _ in range(500000)]
}
transactions_df = pd.DataFrame(transactions_data)
print("transactions_df")

investors_df
assets_df
transactions_df


In [19]:
assets_df.columns

Index(['asset_id', 'investor_id', 'asset_type', 'asset_name', 'purchase_date',
       'purchase_price', 'current_value', 'currency'],
      dtype='object')

In [28]:
import pandas as pd
from langchain.document_loaders  import DataFrameLoader

#from langchain.document_loaders.dataframe import DataFrameLoader


 
# Load data from a Pandas DataFrame using PandasDataFrameLoader
loader = DataFrameLoader(assets_df,page_content_column=['asset_id', 'investor_id', 'asset_type', 'asset_name', 'purchase_date','purchase_price', 'current_value', 'currency'])
documents = loader.load()
 
# Access the content and metadata of each document
for document in documents:
    content = document.page_content
    metadata = document.metadata

 

TypeError: unhashable type: 'list'

In [26]:
documents[2]

Document(page_content='7', metadata={'asset_id': 3, 'asset_type': 'mutual fund', 'asset_name': 'Asset 3', 'purchase_date': '2019-01-07', 'purchase_price': 8454.96, 'current_value': 6145.37, 'currency': 'USD'})